In [112]:
import re

import pandas as pd

import nltk

from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline #make_pipeline, make_union
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import SnowballStemmer
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
# 

from collections import Counter

In [2]:
df_instances = pd.read_json("../data/instances.jsonl", lines=True);
df_truth = pd.read_json("../data/truth.jsonl", lines=True); 

# Display first 5 rows of both
display(df_instances.head())
display(df_truth.head())

,id,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle
0,608310377143799808,[],[Apple's iOS 9 'App thinning' feature will giv...,Tue Jun 09 16:31:10 +0000 2015,['App thinning' will be supported on Apple's i...,'App thinning' will be supported on Apple's iO...,"Apple,gives,gigabytes,iOS,9,app,thinning,featu...",[Paying for a 64GB phone only to discover that...,Apple gives back gigabytes: iOS 9 'app thinnin...
1,609297109095972864,[media/609297109095972864.jpg],[RT @kenbrown12: Emerging market investors are...,Fri Jun 12 09:52:05 +0000 2015,"[Stocks Fall as Investors Watch Central Banks,...",Global investors have yanked $9.3 billion from...,"emerging market,emerging markets,em flows,em i...","[Emerging markets are out of favor., Global in...",Emerging Markets Suffer Largest Outflow in Sev...
2,609504474621612032,[],[U.S. Soccer should start answering tough ques...,Fri Jun 12 23:36:05 +0000 2015,[US to vote for Ali in FIFA election and not B...,A U.S. Senator's scathing letter questioned U....,,"[WINNIPEG, Manitoba â€“ The bubble U.S. Soccer...",U.S. Soccer should start answering tough quest...
3,609748367049105408,[],[How theme parks like Disney World left the mi...,Sat Jun 13 15:45:13 +0000 2015,"[Some 1,000 persons turned out in Albuquerque,...","America's top family vacation spots, like the ...","disney, disney world, disney ticket prices, di...",[When Walt Disney World opened in an Orlando s...,How theme parks like Disney World left the mid...
4,608688782821453824,[media/608688782821453825.jpg],[Could light bulbs hurt your health? One compa...,Wed Jun 10 17:34:49 +0000 2015,[Electric lights have made the world safer and...,One company will put a health notice on all th...,"health, Should there be warning labels on your...",[(CNN)The light bulb always makes the world's ...,Warning labels on your light bulbs


,id,truthClass,truthJudgments,truthMean,truthMedian,truthMode
0,608310377143799808,no-clickbait,"[0.0, 0.6666667, 0.0, 0.33333334000000003, 0.0]",0.200000,0.000000,0.000000
1,609297109095972864,no-clickbait,"[0.6666667, 0.0, 0.0, 0.0, 0.0]",0.133333,0.000000,0.000000
2,609504474621612032,clickbait,"[0.33333334000000003, 0.6666667, 1.0, 0.0, 0.6...",0.533333,0.666667,0.666667
3,609748367049105408,no-clickbait,"[1.0, 0.0, 0.33333334000000003, 0.333333340000...",0.466667,0.333333,0.333333
4,608688782821453824,clickbait,"[1.0, 0.33333334000000003, 0.6666667, 0.333333...",0.666667,0.666667,1.000000


In [3]:
df_truth.groupby('truthClass').count()

,id,truthJudgments,truthMean,truthMedian,truthMode
truthClass,,,,,
clickbait,762,762,762,762,762
no-clickbait,1697,1697,1697,1697,1697


In [129]:
is_number_regex = r"[0-9]+"
is_word_regex = r"[A-Za-z].*"
stemmer = SnowballStemmer("english")

def extract_features(df):
    def extract(df):
        result = dict()
        extract_from_title(df[1]['targetTitle'], result)
        extract_from_article(df[1]['targetParagraphs'], result)
        extract_from_image(df[1]['postMedia'], result)
        return result
        
    def extract_from_title(title, result):
        tiny = title.strip().lower()
        title_words = nltk.word_tokenize(tiny)
        title_words_stem = [stemmer.stem(word) for word in title_words]
        title_words_number_repl = [re.sub(is_number_regex, "[n]", word) for word in title_words_stem]
        twrr_bigram_count = Counter(nltk.bigrams(title_words_number_repl))
        result['title_length'] = len(title)
        result['simple_title_words'] = len(title_words)
        result['title_words'] = len(title.split(' '))
        result['title_question_marks'] = 0 if title.find('?') == -1 else 1
        pos_tag_count = Counter(tag for (word, tag) in nltk.pos_tag(title_words))
        result.update({'pos_tag[{}]'.format(tag): count for tag, count in pos_tag_count.items()})
        result['title_average_word_length'] = len(title) / result['title_words']
        result.update({'title_bigram[{}]'.format(bigram): count for bigram, count in twrr_bigram_count.items()})
        return result
    def extract_from_article(paragraphs, result):
#         tiny = title.strip().lower()
#         title_words = nltk.word_tokenize(tiny)
        result['number_of_paragraphs'] = len(paragraphs)
        entireArticle = ''.join(paragraphs)
        result['article_length'] = len(entireArticle)
        result['article_words'] = len(entireArticle.split(' '))
        result['article_average_word_length'] = len(entireArticle) / len(entireArticle.split(' '))
    
        return result
    def extract_from_image(postMedia, result):
#         tiny = title.strip().lower()
#         title_words = nltk.word_tokenize(tiny)
        result['has_image'] = 1 if len(postMedia) > 0 else 0
    
        return result
    return map(extract, df.iterrows())

# def extract_features_titles(df):
#     def extract_from_title(title):
#         result = dict()
#         tiny = title.strip().lower()
#         title_words = nltk.word_tokenize(tiny)
#         title_words_stem = [stemmer.stem(word) for word in title_words]
#         title_words_number_repl = [re.sub(is_number_regex, "[n]", word) for word in title_words_stem]
#         twrr_bigram_count = Counter(nltk.bigrams(title_words_number_repl))
#         result['title_word_count'] = sum(1 for word in title_words if re.match(is_word_regex, word))
#         result['title_token_count'] = len(title_words)
#         pos_tag_count = Counter(tag for (word, tag) in nltk.pos_tag(title_words))
#         result.update({'pos_tag[{}]'.format(tag): count for tag, count in pos_tag_count.items()})
# #         result.update({'title_bigram[{}]'.format(bigram): count for bigram, count in twrr_bigram_count.items()})
#         return result
#     return map(extract_from_title, df['targetTitle'])

In [5]:
#list(extract_features_titles(df_instances.iloc[:5]['targetTitle']))

In [130]:
clickbaitClassifierNBA = Pipeline([
    ('feature_extraction', FunctionTransformer(extract_features, validate=False)),
    ('encoder', DictVectorizer()),
    ('classifier', MultinomialNB())
])

clickbaitClassifierTree = Pipeline([
    ('feature_extraction', FunctionTransformer(extract_features, validate=False)),
    ('encoder', DictVectorizer()),
    ('classifier', DecisionTreeClassifier())
])

clickbaitClassifierXGB = Pipeline([
    ('feature_extraction', FunctionTransformer(extract_features, validate=False)),
    ('encoder', DictVectorizer()),
    ('classifier', XGBClassifier())
])

clickbaitClassifierSVC = Pipeline([
    ('feature_extraction', FunctionTransformer(extract_features, validate=False)),
    ('encoder', DictVectorizer()),
    ('classifier', LinearSVC(max_iter=4000))
])



dummyClassifier = Pipeline([
    ('feature_extraction', FunctionTransformer(extract_features, validate=False)),
    ('encoder', DictVectorizer()),
    ('classifier', DummyClassifier())
])

In [29]:
# Do a split
merged = pd.merge(df_instances, df_truth, on='id')
X_train, X_test, y_train, y_test = train_test_split(merged, merged['truthClass'])
display(X_train.head())
display(X_test.head())

,id,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle,truthClass,truthJudgments,truthMean,truthMedian,truthMode
1007,609164213412495360,[media/609164213412495360.png],[RT @BuzzFeedNews: Police Shut Down Two Little...,Fri Jun 12 01:04:00 +0000 2015,[Police Shut Down Two Little Girls' Lemonade S...,,,"[â€œIâ€™m still kind of confusedâ€¦â€�, Watch ...",Police Shut Down Two Little Girlsâ€™ Lemonade ...,clickbait,"[0.6666667, 1.0, 0.6666667, 0.0, 0.6666667]",0.600000,0.666667,0.666667
636,608644136648982528,[media/608644136648982528.png],[RT @BuzzFeedMusic: Watch @KaceyMusgraves' â€œ...,Wed Jun 10 14:37:24 +0000 2015,"[Watch Kacey Musgraves' ""Biscuits"" Video And L...",,,[The country star throws a cheeky hoedown in t...,Watch Kacey Musgravesâ€™ â€œBiscuitsâ€� Video ...,clickbait,"[0.6666667, 0.6666667, 0.6666667, 0.0, 0.6666667]",0.533333,0.666667,0.666667
1411,608507562586062848,[media/608507562586062848.jpg],[Secret Service officers reportedly posted at ...,Wed Jun 10 05:34:43 +0000 2015,"[June 9, 2015: A member of the Secret Service ...",Several newly hired Secret Service officers ha...,,[Several newly hired Secret Service officers h...,Secret Service officers reportedly posted at W...,no-clickbait,"[0.0, 0.33333334000000003, 0.33333334000000003...",0.266667,0.333333,0.000000
1556,608934131704168448,[media/608934131704168448.jpg],[RT @commentisfree: Britain has spoken â€“ and...,Thu Jun 11 09:49:45 +0000 2015,"[Philip Hoare, Robin on a snowy branch, Philip...","The robin is brutish, ruthless, and ready to r...","Birds,Wildlife,Animals,Environment,Birdwatchin...",[So weâ€™ve chosen a vicious murdering bully f...,Britain has spoken â€“ and chosen a vicious mu...,no-clickbait,"[0.33333334000000003, 0.33333334000000003, 0.6...",0.533333,0.333333,0.333333
462,610113742785916928,[],"[John Carroll, former editor of @latimes, dies...",Sun Jun 14 15:57:06 +0000 2015,"[The New York Times, Basic, All Access, Home D...","Mr. Carroll, a widely admired newspaper editor...","Deaths (Obituaries),Newspapers,Los Angeles Tim...","[John S. Carroll, a widely admired newspaper e...","John Carroll, Editor Who Reinvigorated The Los...",no-clickbait,"[0.0, 0.0, 1.0, 0.0, 0.0]",0.200000,0.000000,0.000000


,id,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle,truthClass,truthJudgments,truthMean,truthMedian,truthMode
180,609990980691980288,[media/609990980691980288.jpg],[RT @bbcworldservice: Man attacked and killed ...,Sun Jun 14 07:49:17 +0000 2015,"[Radio 1, Radio 1, Radio 1Xtra, Radio 1Xtra, R...",A man in southwest Germany has been attacked...,German killed by elephant,[This app will stop working on 3rd April 2017....,German killed by elephant,clickbait,"[0.6666667, 0.0, 0.6666667, 0.6666667, 1.0]",0.600000,0.666667,0.666667
447,609949762230558720,[media/609949762230558720.jpg],[6 USB-C accessories that'll let your new MacB...,Sun Jun 14 05:05:30 +0000 2015,[Apple's new MacBook has just a single reversi...,Got yourself a new MacBook? These are the USB-...,"apple, uncategorized, tech, gadgets, usb-c, ne...",[USB-C's tiny and reversible design may be the...,6 USB-C accessories that'll let your new MacBo...,clickbait,"[0.6666667, 0.6666667, 0.33333334000000003, 0....",0.666667,0.666667,0.666667
1886,609267488086491136,[],[How Jeb Bushâ€™s campaign ran off course befo...,Fri Jun 12 07:54:23 +0000 2015,"[(Krisztian Bocsi/Bloomberg News), In a speech...",Miscalculations and missteps have undercut his...,,[When asked to pinpoint where Jeb Bushâ€™s pre...,How Jeb Bushâ€™s campaign ran off course befor...,clickbait,"[1.0, 0.33333334000000003, 1.0, 0.333333340000...",0.733333,1.000000,1.000000
1903,608407080853225472,[media/608407080853225472.jpg],[A new type of cholesterol-lowering drug has b...,Tue Jun 09 22:55:26 +0000 2015,[ A mans finger is pricked to test his cholest...,An FDA advisory committee greenlights two expe...,"health, Cholesterol-lowering drugs get FDA adv...",[(CNN)A Food and Drug Administration advisory ...,"Two new, potent cholesterol-lowering drugs get...",clickbait,"[0.0, 1.0, 1.0, 0.6666667, 1.0]",0.733333,1.000000,1.000000
1985,609042117088915456,[media/609042117088915456.jpg],[Kellen Winslow Jr. wasn't ready to end his Tw...,Thu Jun 11 16:58:50 +0000 2015,[],Following New York Jets cornerback Antonio C...,"NFL, Kellen Winslow, Antonio Cromartie, Breaki...",[FollowingÂ New York Jets cornerback Antonio C...,Kellen Winslow Jr. Challenges Antonio Cromarti...,no-clickbait,"[0.0, 0.33333334000000003, 0.33333334000000003...",0.333333,0.333333,0.333333


In [131]:
clickbaitClassifierNBA.fit(X_train, y_train);
clickbaitClassifierTree.fit(X_train, y_train);
clickbaitClassifierSVC.fit(X_train, y_train);
clickbaitClassifierXGB.fit(X_train, y_train);
dummyClassifier.fit(X_train, y_train);

c:\users\x8uur\.virtualenvs\irnlpproject-v-e943d6\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [71]:
from sklearn.metrics import precision_score, recall_score

In [132]:
pred_tree = clickbaitClassifierTree.predict(X_test)
pred_nb = clickbaitClassifierNBA.predict(X_test)
pred_svc = clickbaitClassifierSVC.predict(X_test)
pred_xgb = clickbaitClassifierXGB.predict(X_test)
pred_dummy = dummyClassifier.predict(X_test)


In [133]:
preds = [pred_tree, pred_nb, pred_svc, pred_xgb, pred_dummy]

truthmap = [['clickbait', 'no-clickbait'].index(item) for item in y_test]

predsmap = [[['clickbait', 'no-clickbait'].index(item) for item in pred ] for pred in preds]


precisions = [precision_score(truthmap, pred) for pred in predsmap]
recalls = [recall_score(truthmap, pred) for pred in predsmap]
pd.DataFrame({"Classifier": ["DecisionTree", "NaiveBayes", "SVC", "XGBoost", "Dummy"], 
               "Precision": precisions,
               "Recall": recalls})

,Classifier,Precision,Recall
0,DecisionTree,0.751606,0.822014
1,NaiveBayes,0.741445,0.456674
2,SVC,0.783333,0.330211
3,XGBoost,0.777419,0.564403
4,Dummy,0.678984,0.688525


,Classifier,Precision,Recall
0,DecisionTree,0.771670,0.854801
1,NaiveBayes,0.715532,0.960187
2,SVC,0.770161,0.894614
3,XGBoost,0.745387,0.946136
4,Dummy,0.699519,0.681499


In [121]:
#tr = clickbaitClassifierTree.named_steps['classifier']
#dv = clickbaitClassifierTree.named_steps['encoder']
tr = clickbaitClassifierXGB.named_steps['classifier']
dv = clickbaitClassifierXGB.named_steps['encoder']

dfFeatureImportance = pd.DataFrame(list(zip(dv.feature_names_, tr.feature_importances_)))
dfFeatureImportance.sort_values(1, ascending=False)

,0,1
10,pos_tag[DT],0.050623
6,pos_tag[.],0.033212
819,"title_bigram[('-', 'busi')]",0.030776
16994,title_word_count,0.028542
8301,"title_bigram[('is', 'the')]",0.027896
41,pos_tag[WRB],0.025297
9,pos_tag[CD],0.024289
16993,title_token_count,0.024178
24,pos_tag[PRP],0.023414
20,pos_tag[NN],0.023109
